# ACME Automated Certificate Management with Let's Encrypt and Certbot

This notebook demonstrates how to set up an automated certificate management environment using the ACME protocol with Let's Encrypt and Certbot.

Note: This notebook uses bash commands. Make sure you have the bash kernel installed for Jupyter.
You can install it using: pip install bash_kernel
Then run: python -m bash_kernel.install

## 1. Install Certbot
(Uncomment the appropriate line for your system)

In [ ]:
# For Ubuntu/Debian:
# !sudo apt-get update
# !sudo apt-get install certbot

# For CentOS/RHEL:
# !sudo yum install certbot

# For macOS (using Homebrew):
# !brew install certbot

In [ ]:
%%bash
# 2. Obtain a certificate (example for Apache)
# Replace 'example.com' with your domain name
sudo certbot --apache -d example.com -d www.example.com

# 3. Test automatic renewal
sudo certbot renew --dry-run

# 4. Set up a cron job for automatic renewal
# This command adds a cron job that runs twice daily
(crontab -l 2>/dev/null; echo "0 0,12 * * * /usr/bin/certbot renew --quiet") | crontab -

# 5. Create a simple renewal script
cat << EOF > renew_certs.sh
#!/bin/bash

# Renew the certificates
certbot renew --quiet

# Restart web server to pick up new certificates
systemctl restart apache2  # or 'nginx' if you're using Nginx

# Log the renewal attempt
echo "Certificate renewal attempted on $(date)" >> /var/log/cert_renewal.log
EOF

chmod +x renew_certs.sh

# 6. Set up a more detailed cron job using the script
(crontab -l 2>/dev/null; echo "0 0,12 * * * /path/to/renew_certs.sh") | crontab -

# 7. Implement post-renewal hooks
# Create a script to be executed after successful renewal
cat << EOF > post_renewal_hook.sh
#!/bin/bash

# Restart web server
systemctl restart apache2  # or 'nginx' if you're using Nginx

# Send notification
echo "Certificates were successfully renewed on $(date)" | mail -s "Certificate Renewal Notification" admin@example.com
EOF

chmod +x post_renewal_hook.sh

# Configure Certbot to use the post-renewal hook
sudo certbot renew --deploy-hook /path/to/post_renewal_hook.sh

# 8. Monitor certificate expiration
# Create a script to check certificate expiration
cat << EOF > check_cert_expiration.sh
#!/bin/bash

DOMAIN="example.com"
DAYS_THRESHOLD=30

expiration_date=$(openssl s_client -servername $DOMAIN -connect $DOMAIN:443 2>/dev/null | openssl x509 -noout -enddate | cut -d= -f2)
expiration_epoch=$(date -d "$expiration_date" +%s)
current_epoch=$(date +%s)
days_until_expiration=$(( ($expiration_epoch - $current_epoch) / 86400 ))

if [ $days_until_expiration -le $DAYS_THRESHOLD ]; then
    echo "Warning: Certificate for $DOMAIN will expire in $days_until_expiration days!" | mail -s "Certificate Expiration Warning" admin@example.com
fi
EOF

chmod +x check_cert_expiration.sh

# Add this script to crontab to run daily
(crontab -l 2>/dev/null; echo "0 0 * * * /path/to/check_cert_expiration.sh") | crontab -

# 9. Backup Certbot configuration and certificates
# Create a backup script
cat << EOF > backup_certs.sh
#!/bin/bash

BACKUP_DIR="/path/to/backup/directory"
DATE=$(date +"%Y%m%d")

# Backup Certbot configuration
tar -czf $BACKUP_DIR/certbot_config_$DATE.tar.gz /etc/letsencrypt

# Backup Apache/Nginx configuration (uncomment appropriate line)
# tar -czf $BACKUP_DIR/apache_config_$DATE.tar.gz /etc/apache2
# tar -czf $BACKUP_DIR/nginx_config_$DATE.tar.gz /etc/nginx

echo "Backup completed on $(date)" >> $BACKUP_DIR/backup.log
EOF

chmod +x backup_certs.sh

# Add this script to crontab to run weekly
(crontab -l 2>/dev/null; echo "0 0 * * 0 /path/to/backup_certs.sh") | crontab -

# Note: Remember to replace 'example.com', '/path/to/', and 'admin@example.com' with your actual values.